## 1. Filter for Toponym Occurrances

1. Download a year of comments
2. load country comments set
3. decompress country comments
4. find occurrances of set elements in year and check new file size

In [2]:

import json
import pandas as pd
import time 
import os

#decompressing
import bz2
import lzma
import zstandard as zstd

#NLP
from tqdm import tqdm_notebook
import spacy
from collections import Counter
nlp = spacy.load("en_core_web_sm")

#### Create cities list

### Get one document at a time

In [16]:
city_df = pd.read_csv('city_variations.csv')

#filter to top 50. This should fix the 'Nice' occurrance issue
#add lowercase to set
top50_df = city_df.head(50)

cities = set(list(top50_df['eng'])+list(top50_df['wiki'])+list(top50_df['local']))

low = []

for i in cities:
    low.append(i.lower())

cities = cities | set(low)

In [17]:
col_filter = [ 'created_utc', 'score', 'subreddit',
        'link_id', 'subreddit_id', 'body']

#### bz2 to .txt

In [320]:
files = [
 'RC_2015-09.bz2',
 'RC_2015-10.bz2',
 'RC_2015-11.bz2',
 'RC_2015-12.bz2']

In [1]:
def bz2_to_df_1(file, open_fp, write_fp, cities):
    counter = 0
    with bz2.open(open_fp+file, "rt") as bzinput:
        lines = []
        with open(write_fp+file[:-4]+'.txt', "w", encoding='utf-8') as f:
            for i, line in enumerate(bzinput):
                post = json.loads(line)
                for top in cities:
                    #filter before tokenizing
                    if top in post['body']:
                        #tokenize
                        words = post['body'].split()
                        if top in words:
                            #idx = int(post['body'].find(top))
                            #upper = idx+len(top)+3
                            #lower = idx - 3
                            #neighborhood = post['body'][lower:upper]
                            
                            
                            
                            for i in col_filter:
                                f.write(str(post[i]).replace(',' , '').replace('\n', ' ') + ',')
                                
                            f.write('\n--------\n')
                            break

    return 

#### lzma to .txt

In [313]:
files = ['RC_2018-04.xz','RC_2018-05.xz','RC_2018-06.xz', 'RC_2018-07.xz']

In [310]:

def lz_to_df_1(file, open_fp, write_fp, cities):
    counter = 0
    with lzma.open(open_fp+file, "rt") as lzinput:
        lines = []
        with open(write_fp+file[:-3]+'.txt', "w", encoding='utf-8') as f:
            for i, line in enumerate(lzinput):
                post = json.loads(line)
                for top in cities:
                    #filter before tokenizing
                    if top in post['body']:
                        #tokenize
                        words = post['body'].split()
                        if top in words:
                            #idx = int(post['body'].find(top))
                            #upper = idx+len(top)+3
                            #lower = idx - 3
                            #neighborhood = post['body'][lower:upper]
                            
                            
                            
                            for i in col_filter:
                                f.write(str(post[i]).replace(',' , '').replace('\n', ' ') + ',')
                                
                            f.write('\n--------\n')
                            break

    return 

In [314]:
for file in files:

    start = time.time()
    write_fp = r"E:\\citynet cooccurrances\\"
    lz_to_df_1(file, open_fp, write_fp, cities)

    end = time.time() 
    print(str(end-start)+' seconds')

6584.823339223862 seconds
4865.527549266815 seconds
5339.1321902275085 seconds
5911.253631591797 seconds


#### zstd to .txt

In [ ]:
files = []

In [ ]:

def zst_to_df_1(file, open_fp, write_fp, cities):
    counter = 0
    with zst.open(open_fp+file, "rt") as zstinput:
        lines = []
        with open(write_fp+file[:-3]+'.txt', "w", encoding='utf-8') as f:
            for i, line in enumerate(zstinput):
                post = json.loads(line)
                for top in cities:
                    #filter before tokenizing
                    if top in post['body']:
                        #tokenize
                        words = post['body'].split()
                        if top in words:
                            #idx = int(post['body'].find(top))
                            #upper = idx+len(top)+3
                            #lower = idx - 3
                            #neighborhood = post['body'][lower:upper]
                            
                            
                            
                            for i in col_filter:
                                f.write(str(post[i]).replace(',' , '').replace('\n', ' ') + ',')
                                
                            f.write('\n--------\n')
                            break

    return 

In [ ]:
for file in files:

    start = time.time()
    write_fp = r"E:\\citynet cooccurrances\\"
    zst_to_df_1(file, open_fp, write_fp, cities)

    end = time.time() 
    print(str(end-start)+' seconds')

#### zst decompression streaming for files too large

In [326]:
files = []

In [ ]:
for file in files:
    with open(open_fp+file, 'rb') as fh:
        dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
        with dctx.stream_reader(fh) as reader:
            previous_line = ""

            with open(write_fp+file[:-4]+'.txt', "w", encoding='utf-8') as f:

                while True:
                    chunk = reader.read(2**24)  # 16mb chunks
                    if not chunk:
                        break

                    string_data = chunk.decode('utf-8')
                    lines = string_data.split("\n")
                    for i, line in enumerate(lines[:-1]):
                        if i == 0:
                            line = previous_line + line
                        try:
                            post = json.loads(line)
                        except:
                            continue

                        for top in cities:
                            #filter before tokenizing
                            if top in post['body']:
                                #tokenize
                                words = post['body'].split()
                                if top in words:
                                    #idx = int(post['body'].find(top))
                                    #upper = idx+len(top)+3
                                    #lower = idx - 3
                                    #neighborhood = post['body'][lower:upper]



                                    for i in col_filter:
                                        f.write(str(post[i]).replace(',' , '').replace('\n', ' ') + ',')

                                    f.write('\n--------\n')
                                    break

                previous_line = lines[-1]

#### Speed Test

-loop through upper and lower 

-convert post to lower and only loop lower cities

-find cooccurrances as well as occurrances

-decision: loop through upper and lower